In [30]:
import twitter
import requests

In [23]:
class Twitter(object):
    def __init__(self):
        consumer_key = 'LFTXnbNVhCtLV2T2dG0yA'
        consumer_secret = '2DX6L81tUHP8Clxz8FvuXcaMvHTAXuUkXbzz7y6U'
        access_token = '77452634-avWq03PwpORlOtKoa4ctg082fTdFlBbKT0ihC8XZw'
        access_token_secret = 'cnopinzlsPRi6zBOMGdAVf0EuMM97eCQF5UPoyTyY'

        self.api = twitter.Api(consumer_key=consumer_key,
                               consumer_secret=consumer_secret,
                               access_token_key=access_token,
                               access_token_secret=access_token_secret)
        
    def get_tweet(self, id):
        return self.api.GetStatus(id).AsDict()

In [35]:
def get_geoname_data(geoname):
    return requests.get(u'http://api.geonames.org/postalCodeSearchJSON?placename={0}&maxRows=100&username=demo'.format(geoname)).json()